In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install -U transformers datasets torch rapidfuzz opencv-python jiwer pillow torchvision torchaudio --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [ ]:
!pip uninstall -y pillow
!pip install "pillow==11.3.0" --no-cache-dir --quiet


from PIL import Image
print(Image.__version__)


!pip install transformers datasets torch rapidfuzz opencv-python jiwer pillow==11.3.0 --quiet


Found existing installation: pillow 11.3.0
Uninstalling pillow-11.3.0:
  Successfully uninstalled pillow-11.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.7 MB/s eta 0:00:00
11.3.0


In [1]:
from transformers import AutoTokenizer


# =======================================
# Medicine OCR + Correction + Information Pipeline
# CLEAN & CORRECTED NOTEBOOK (Same filenames preserved)


# =======================================
# STEP 1: Imports
# =======================================
import random
import torch
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from PIL import Image
from datasets import Dataset
from rapidfuzz import process, fuzz
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    pipeline,
    TrOCRProcessor,
    VisionEncoderDecoderModel
)

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =======================================
# STEP 2: Load medicine dataset (names)
# =======================================
df = pd.read_csv("/content/A_Z_medicines_dataset_of_India.csv")

# =======================================
# STEP 3: OCR-style noise generator (cleaned)
# =======================================
def add_noise(text, noise_factor=0.4):
    substitutions = {
        "0": "O", "O": "0",
        "1": "I", "I": "1",
        "5": "S", "S": "5",
        "8": "B", "B": "8",
        "c": "e", "e": "c",
        "l": "I"
    }

    text = list(text)
    num_noises = max(1, int(len(text) * noise_factor))

    for _ in range(num_noises):
        op = random.choice(["swap", "delete", "insert", "replace", "duplicate", "caseflip", "space"])
        i = random.randint(0, len(text) - 1)

        if op == "swap" and i < len(text) - 1:
            text[i], text[i + 1] = text[i + 1], text[i]
        elif op == "delete":
            text[i] = ""
        elif op == "insert":
            text.insert(i, random.choice("abcdefghijklmnopqrstuvwxyz"))
        elif op == "replace":
            text[i] = substitutions.get(text[i], random.choice("abcdefghijklmnopqrstuvwxyz"))
        elif op == "duplicate":
            text.insert(i, text[i])
        elif op == "caseflip":
            text[i] = text[i].upper() if text[i].islower() else text[i].lower()
        elif op == "space":
            text.insert(i, " ")

    return "".join(text).strip()

# =======================================
# STEP 4: Create synthetic dataset
# =======================================
clean_names = df['name'].dropna().unique().tolist()[:9000]

data = [
    {"input_text": add_noise(name.lower()), "target_text": name.lower()}
    for name in clean_names
]

dataset = Dataset.from_list(data)
print("✅ Synthetic dataset created:", len(dataset))

# =======================================
# STEP 5: Load ByT5 model
# =======================================
BASE_MODEL = "google/byt5-small"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL).to(DEVICE)

# =======================================
# STEP 6: Tokenization (padding fixed)
# =======================================
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=64,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        examples["target_text"],
        max_length=64,
        padding="max_length",
        truncation=True
    ).input_ids

    labels = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs


tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)

# =======================================
# STEP 7: Training
# =======================================
training_args = TrainingArguments(
    output_dir="/content/byt5-medicine-corrector",
    eval_strategy="no",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    optim="adafactor",
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
trainer.save_model("/content/byt5-medicine-corrector")
print("✅ Model saved at /content/byt5-medicine-corrector")

# =======================================
# STEP 8: Load fine-tuned corrector (SAFE)
# =======================================
MODEL_PATH = "/content/byt5-medicine-corrector"

corrector_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
corrector_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH, local_files_only=True).to(DEVICE)
corrector_model.eval()


def correct_text(text):
    inputs = corrector_tokenizer(text.lower(), return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = corrector_model.generate(**inputs, max_length=64)
    return corrector_tokenizer.decode(outputs[0], skip_special_tokens=True)

# =======================================
# STEP 9: Load Medicine details dataset
# =======================================
df1 = pd.read_csv("/content/Medicine_Details.csv")
df1.columns = [c.strip().replace(" ", "_") for c in df1.columns]

# =======================================
# STEP 10: Fuzzy match + explanation
# =======================================

def get_closest_medicine(query, df, limit=3):
    names = df["Medicine_Name"].tolist()
    return process.extract(query, names, scorer=fuzz.WRatio, limit=limit)


def generate_drug_explanation(drug_name, uses, side_effects, composition=None):
    return (
        f"The medicine '{drug_name}' is composed of {composition or 'active ingredients'}. "
        f"It is used for {uses}. Common side effects include {side_effects}. "
        "This information is for educational purposes only."
    )


generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=80,
    device=0 if DEVICE == "cuda" else -1
)


def make_readable_explanation(text):
    return generator(f"Rephrase simply and professionally: {text}")[0]['generated_text']


def get_drug_info(ocr_text):
    ocr_text = ocr_text.lower().strip()
    matches = get_closest_medicine(ocr_text, df1)
    best_match, score, idx = matches[0]
    record = df1.iloc[idx]

    explanation = generate_drug_explanation(
        record["Medicine_Name"],
        record["Uses"],
        record["Side_effects"],
        record.get("Composition")
    )

    return {
        "OCR Input": ocr_text,
        "Matched Medicine": record["Medicine_Name"],
        "Match Confidence": score,
        "Generated Explanation": make_readable_explanation(explanation)
    }

# =======================================
# STEP 11: OCR using TrOCR (PRINTED)
# =======================================
from google.colab import files
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(3.0, (8, 8))
gray = clahe.apply(gray)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed").to(DEVICE)

trocr_model.eval()
img_pil = Image.fromarray(thresh).convert("RGB")

pixel_values = processor(images=img_pil, return_tensors="pt").pixel_values.to(DEVICE)

with torch.no_grad():
    ids = trocr_model.generate(pixel_values)

ocr_output = processor.batch_decode(ids, skip_special_tokens=True)[0]

# =======================================
# STEP 12: Final pipeline
# =======================================
corrected = correct_text(ocr_output)
result = get_drug_info(corrected)

print("🧠 Raw OCR:", ocr_output)
print("✨ Corrected:", corrected)
print("✅ Medicine:", result["Matched Medicine"], f"({result['Match Confidence']:.1f}%)")
print("💬 Explanation:", result["Generated Explanation"])


ModuleNotFoundError: No module named 'rapidfuzz'

In [2]:
!pip install \
  transformers \
  datasets \
  torch \
  rapidfuzz \
  opencv-python \
  jiwer \
  pillow==11.3.0 \
  torchvision \
  torchaudio \
  --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.6 MB/s eta 0:00:00
